## 메타데이터 불러오기

- MovieLens 25M Dataset(https://grouplens.org/datasets/movielens/25m/) 기준
- 위 데이터에 없는 keyword, poster_path, 감독, 출연진 등 정보들 크롤링

In [ ]:
import pandas as pd
import json

metadata_df = pd.read_json('../movie_dataset_public_final/raw/metadata.json', lines=True)

## IMDB 크롤링

In [ ]:
!pip install imdbpy

     |████████████████████████████████| 298 kB 23.6 MB/s 


In [ ]:
from imdb import IMDb
import requests

In [ ]:
ia = IMDb()

id2metadata = {}

for idx, imdbId in tqdm(enumerate(metadata_df['imdbId']), total = len(metadata_df)):
    movie = ia.get_movie(imdbId)
    temp_metadata = {}
    for key in ['title', 'year', 'genre', 'runtimes', 'countries', 'languages', 'box office', 'kind', 'rating', 'votes', 'plot', 'synopsis', 'cover url', 'full-size cover url']:
        try: 
            temp_metadata[key] = movie[key]
        except:
            temp_metadata[key] = 'No'
    id2metadata[imdbId] = temp_metadata
    
    # if (idx+1) % 1000 == 0:
    #     with open('IMDB.json', 'w') as f:
    #         json.dump(id2metadata, f)
    #     pd.DataFrame(id2metadata).transpose().to_csv('IMDB.csv')

# TMDB 크롤링

In [ ]:
my_key = 'TMDB API key'

total_info = []
for idx, movie_id in tqdm(enumerate(tmdb_id), total=len(tmdb_id)):

    movie_url= f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={my_key}"
    keyword_url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={my_key}"
    credit_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={my_key}"

    movie_response = requests.get(movie_url)
    keyword_response = requests.get(keyword_url)
    credit_response = requests.get(credit_url)

    movie_dict = movie_response.json()
    keyword_dict = keyword_response.json()
    credit_dict = credit_response.json()

    info = {**movie_dict, **keyword_dict, **credit_dict}

    # total_info.append(info)
    # if idx % 1000 == 0:
    #     pd.DataFrame(total_info).to_json('tmdb_crawling.json')
    #     pd.DataFrame(total_info).to_csv('tmdb_crawling.csv')


  0%|          | 0/62316 [00:00<?, ?it/s]